In [ ]:
Josh Gilkinson
DSC Week 10 

In [17]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def load_data():
    movies = pd.read_csv(r'C:\Users\jgilk\OneDrive\Desktop\Data Science\DSC630\movies_small.csv')
    ratings = pd.read_csv(r'C:\Users\jgilk\OneDrive\Desktop\Data Science\DSC630\ratings_small.csv')
    return movies, ratings

#Merge the two datasets
def preprocess_data(movies, ratings):
    movie_ratings = ratings.groupby('movieId').mean()['rating']
    movies = movies.merge(movie_ratings, on='movieId', how='left')
    return movies

#I used Cosine Similarity and TF-IDF to convert genre into numerical
# Values and then compairing them 
def compute_similarity(movies):
    # Use TF-IDF Vectorizer on movie genres and remove stop words
    tfidf = TfidfVectorizer(stop_words="english")
    tfidf_matrix = tfidf.fit_transform(movies['genres'].fillna(''))
    sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return sim_matrix

# Now that the movies are scored, the top 10 most similar movies
# to my test movie are returned. I had an issue with the test
# movie being returned as one of the 10 so I had to exclude that 
def recommend_movies(movie_title, movies, sim_matrix):
    idx = movies[movies['title'] == movie_title].index[0]
    sim_scores = list(enumerate(sim_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    recommended_indices = [i[0] for i in sim_scores if i[0] != idx][:10]  
    return movies.iloc[recommended_indices]['title'].tolist()

movies, ratings = load_data()
movies = preprocess_data(movies, ratings)
sim_matrix = compute_similarity(movies)

# Test Movie
user_movie = "In the Mouth of Madness (1995)"
recommendations = recommend_movies(user_movie, movies, sim_matrix)
print("Recommended movies:", recommendations)


Recommended movies: ['Relative Fear (1994)', 'Tales from the Crypt Presents: Demon Knight (1995)', 'Thinner (1996)', 'Relic, The (1997)', 'Halloween: The Curse of Michael Myers (Halloween 6: The Curse of Michael Myers) (1995)', 'Believers, The (1987)', 'Birds, The (1963)', 'Body Parts (1991)', 'Candyman (1992)', 'Nightmare on Elm Street, A (1984)']


In [ ]:
# Sources: 
# https://www.kaggle.com/code/ibtesama/getting-started-with-a-movie-recommendation-system
# https://medium.com/@abhishekshaw020/python-project-building-a-movie-recommendation-system-175f5f32aa10